In [1]:
from freedom import freedom
from data import data
from analyze import analyze
import matplotlib.pyplot as plt
import seaborn as sb
import plotly as ply
import geopandas as geo
import pandas as pd
import numpy as np

In [7]:
analyze(data, "c-Brazil")

,GDP per capita (current US$),"Agriculture, forestry, and fishing, value added (% of GDP)",Food production index (2004-2006 = 100),Arable land (hectares per person),Surface area (sq. km),Rural population (% of total population),Access to electricity (% of population),CO2 emissions (metric tons per capita),Urban population,"Population, total",...,Lending interest rate (%),Real interest rate (%),"Stocks traded, total value (% of GDP)","Listed domestic companies, total",Domestic credit to private sector (% of GDP),Bank capital to assets ratio (%),"Official exchange rate (LCU per US$, period average)",Time required to start a business (days),New businesses registered (number),Total tax rate (% of commercial profits)
1963,NaN,15.1257,16.76,0.329723,8.51577e+06,50.922,NaN,0.705321,3.87e+07,7.8854e+07,...,NaN,NaN,NaN,NaN,14.8757,NaN,2.48869e-13,NaN,NaN,NaN
1964,NaN,15.8383,17.61,0.332641,8.51577e+06,49.941,NaN,0.698987,4.06322e+07,8.11687e+07,...,NaN,NaN,NaN,NaN,13.3653,NaN,5.42024e-13,NaN,NaN,NaN
1965,NaN,16.6744,19.64,0.335337,8.51577e+06,48.963,NaN,0.675447,4.26149e+07,8.3498e+07,...,NaN,NaN,NaN,NaN,13.4923,NaN,8.21655e-13,NaN,NaN,NaN
1966,NaN,13.3153,19.13,0.337847,8.51577e+06,47.984,NaN,0.74914,4.46494e+07,8.58378e+07,...,NaN,NaN,NaN,NaN,12.4336,NaN,9.60713e-13,NaN,NaN,NaN
1967,NaN,13.0131,20.59,0.340169,8.51577e+06,47.007,NaN,0.750561,4.67353e+07,8.81914e+07,...,NaN,NaN,NaN,NaN,14.4392,NaN,1.1526e-12,NaN,NaN,NaN
1968,NaN,11.8265,21.27,0.353368,8.51577e+06,46.03,NaN,0.854946,4.88736e+07,9.05571e+07,...,NaN,NaN,NaN,NaN,16.6777,NaN,1.4696e-12,NaN,NaN,NaN
1969,NaN,11.0761,21.97,0.355087,8.51577e+06,45.06,NaN,0.90725,5.10585e+07,9.29351e+07,...,NaN,NaN,NaN,NaN,26.5183,NaN,1.76336e-12,NaN,NaN,NaN
1970,NaN,10.3736,23.42,0.367158,8.51577e+06,44.091,NaN,0.98358,5.32963e+07,9.53268e+07,...,NaN,NaN,NaN,NaN,30.8921,NaN,1.98785e-12,NaN,NaN,NaN
1971,NaN,11.0142,23.91,0.378598,8.51577e+06,43.106,NaN,1.05021,5.56019e+07,9.7729e+07,...,NaN,NaN,NaN,NaN,37.2984,NaN,2.28822e-12,NaN,NaN,NaN
1972,NaN,11.069,25.4,0.379455,8.51577e+06,42.121,NaN,1.14199,5.79621e+07,1.00144e+08,...,NaN,NaN,NaN,NaN,41.7629,NaN,2.56796e-12,NaN,NaN,NaN


In [ ]:
analyze(data, "GDP per capita (current US$)")

In [ ]:
df2 = freedom[[2017]].drop('Korea, Rep.', 0)

In [ ]:
df1 = np.log10(df2.join(analyze(data, 2017,["GDP per capita (current US$)"])))
df1.columns = ['x', 'y']
df1.plot('x', 'y', kind='scatter')
plt.show()

In [ ]:
def mmm(x=3,age=2017, ind="GDP per capita (current US$)"):
    df1 = analyze(data, age, [ind]).sort_values([ind])
    lmd1 = int(len(df1)/2)-int(x/2)
    df1.iloc[list(range(0,x))+list(range(lmd1, lmd1+x))+list(range(-x,0))].plot(kind="bar")
    plt.show()

In [ ]:
mmm()

In [ ]:
pd.DataFrame?

In [ ]:
x = 2
df1 = freedom[[2017]].drop('Korea, Rep.', 0).dropna().sort_values([2017])
lmd1 = int(len(df1)/2)
df1.iloc[list(range(0,x))+list(range(lmd1-int(x/2), lmd1+1+int(x/2)))+list(range(-x,0))].plot(kind="bar")
plt.show()

In [ ]:
df1 = analyze(data, 2017, ["GDP per capita (current US$)"])
df1.groupby(["GDP per capita (current US$)", pd.cut(df1.distances, 2)]).value.mean()



In [ ]:
def analyze(data, ageorcountryorind, indice=False, rm=True, log=False, disp=False):
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    from sklearn import linear_model
    countries = [i.replace("\n", "") for i in open("countries.txt")]
    
    if type(ageorcountryorind) == int:
        country = pd.DataFrame(data[0][str(ageorcountryorind)], columns=[data[0]['Indicator Name'][0]])
        for k,i in enumerate(data[1:]):
            country = country.join(pd.Series(i[str(ageorcountryorind)], name=data[k+1]["Indicator Name"][0]), how="outer")
        df = country.reindex(countries) if rm else country
    
    elif ageorcountryorind[0:2] == "c-":
        ages = pd.DataFrame(data[0].T[ageorcountryorind[2:]][4:], columns=[data[0]["Indicator Name"][0]])
        for k,i in enumerate(data[1:]):
            ages = ages.join(pd.Series(i.T[ageorcountryorind[2:]][4:], name=data[k+1]["Indicator Name"][0]), how="outer")
        df = ages
        
    else:
        for i in data:
            if i['Indicator Name'][0] == ageorcountryorind:
                saida = i.drop("Indicator Name", axis=1)
                break
        df = saida.reindex(countries) if rm else saida

    if indice==False:
        return df.dropna(how='all')
    elif log:
        df = np.log(df[indice].dropna()) 
        if disp:
            y = np.array(df[indice[0]]).reshape(-1, 1)
            x = np.array(df[indice[1]]).reshape(-1, 1)
            model = linear_model.LinearRegression().fit(x, y)
            plt.plot(x, model.predict(x), color='red', linewidth=2.)
            plt.scatter(y=y, x=x, color='blue', s=50, alpha=.5)
            plt.title('Reta ajustada log-log')
            plt.xlabel(indice[0])
            plt.ylabel(indice[1])
            plt.show()
        else:
            return df
    else:
        df = df[indice].dropna()
        if disp:
            y = df[indice[0]]
            x = df[indice[1]]
            plt.scatter(y=y, x=x, color='blue', s=50, alpha=.5)
            plt.title('Gráfico de dispersao')
            plt.xlabel(indice[0])
            plt.ylabel(indice[1])
            plt.show()
        else:
            return df
        